In [0]:
%pip install nltk

Python interpreter will be restarted.
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached click-8.1.2-py3-none-any.whl (96 kB)
  Using cached regex-2022.3.15-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (764 kB)
Python interpreter will be restarted.


In [0]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
import string
import re
import numpy as np



stop_words = set(stopwords.words('english'))

def tokenize(sentence):

    t= re.split(r"\s+|\\", sentence)
    filtered_t = []
    temp = t[0]
    
    size = len([word for word in t if not word in stop_words]) - 1       
    for i in range(1,len(t)):
        if(t[i].lower() not in stop_words):
            filtered_t.append([(re.sub('[^A-Za-z0-9]+', '', t[i].lower()),temp, size), 1])
            
    return (filtered_t)
  
inp = sc.textFile("/FileStore/tables/plot_summaries.txt") # input
# /FileStore/tables/movie_metadata.tsv
df = spark.read.csv("/FileStore/tables/movie_metadata.tsv", sep=r'\t', header=False)
df = df.rdd.map(lambda x : (x[0],x[2]))

clean_input = inp.map(tokenize)
total_doc = clean_input.count()
word_count = clean_input.flatMap(lambda x: x).reduceByKey(lambda x,y: x+y )
#word_count = clean_input.flatMap(lambda x: x).reduceByKey(lambda x, y:(float(x[0])+ float(y[0]), x[1]+ y[1])).mapValues(lambda x: float(x[0])/float(x[1]))



---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<command-3986918536345065> in <module>
----> 1 import nltk
      2 nltk.download('stopwords')
      3 from nltk.corpus import stopwords
      4 import string
      5 import re

/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py in import_patch(name, globals, locals, fromlist, level)
    165             # Import the desired module. If you’re seeing this while debugging a failed import,
    166             # look at preceding stack frames for relevant error information.
--> 167             original_result = python_builtin_import(name, globals, locals, fromlist, level)
    168 
    169             is_root_import = thread_local._nest_level == 1

ModuleNotFoundError: No module named 'nltk'

In [0]:
#clean_input.take(1)
#clean_input.flatMap(lambda x: x).take(10)
#clean_input.flatMap(lambda x: x).reduceByKey(lambda x,y: x+y).take(10)

#clean_input.map(lambda x: x[3][2]).take(1)
#clean_input.take(1)
#total_doc
word_count.take(10)

Out[111]: [(('lovehate', '23890098', 15), 1),
 (('way', '31186339', 479), 1),
 (('presented', '31186339', 479), 1),
 (('tells', '31186339', 479), 1),
 (('friend', '20663735', 303), 2),
 (('lemon', '2231378', 517), 1),
 (('lindy', '595909', 223), 7),
 (('public', '595909', 223), 2),
 (('changes', '1952976', 444), 1),
 (('pick', '1952976', 444), 1)]

In [0]:
#given = sc.parallelize(['hand','death'])
given = sc.parallelize(['death'])
given = given.collect()

In [0]:
word_count.filter(lambda x: x[0][0] == "shlykov" ).count()

Out[26]: 1

In [0]:
scount = word_count.filter(lambda x: x[0][0] == "death" )

In [0]:
#scount[0][0][2]
scount.map(lambda x: [x[0][1],x[1], x[0][2]]).collect()
#scount.collect()

Out[90]: [['9687588', 1, 240],
 ['22641776', 2, 229],
 ['164502', 2, 373],
 ['26048721', 1, 121],
 ['11502520', 1, 500],
 ['35773659', 1, 54],
 ['11035177', 1, 66],
 ['19068826', 1, 42],
 ['31106274', 1, 253],
 ['12521963', 1, 79],
 ['1011144', 1, 192],
 ['16731473', 1, 347],
 ['15749688', 1, 62],
 ['20028957', 1, 363],
 ['4298222', 1, 1039],
 ['2651130', 4, 318],
 ['466340', 1, 589],
 ['8172107', 1, 71],
 ['17705805', 1, 230],
 ['24653', 1, 379],
 ['6651585', 1, 368],
 ['3195590', 1, 250],
 ['9885736', 1, 658],
 ['11623872', 1, 221],
 ['5056637', 2, 527],
 ['4736692', 1, 56],
 ['8888296', 2, 426],
 ['4198198', 5, 211],
 ['2571604', 1, 511],
 ['17183671', 1, 85],
 ['2951887', 1, 85],
 ['14149884', 1, 472],
 ['1179229', 1, 446],
 ['2232503', 1, 580],
 ['11410391', 1, 625],
 ['7154803', 4, 1013],
 ['1519886', 2, 612],
 ['28963028', 1, 48],
 ['8603532', 1, 422],
 ['3598844', 2, 336],
 ['10542198', 1, 130],
 ['6745617', 1, 587],
 ['15687340', 1, 204],
 ['22540513', 1, 126],
 ['34515864', 1

In [0]:
ans = []

for i in given:
    scount = word_count.filter(lambda x: x[0][0] == i)
    ndoc = scount.count()
    temp = np.log(float(total_doc)/ndoc)
    print(ndoc)
    print(total_doc)
    movies = scount.map(lambda x: [x[0][1], float(x[1]/x[0][2])*temp]).sortBy(lambda x: -x[1]).take(10)
    
    m = sc.parallelize(movies).join(df).map(lambda x: x[1][1])
    ans.append((i,m.collect()))

7015
42306


In [0]:
#word_count.map(lambda x: x[0][1]).take(1)
word_count.map(lambda x: x).reduceByKey(lambda x,y: x[0][1]+y[0][1] ).take(5)

Out[99]: [(('major', '5786261', 277), 1),
 (('person', '19332675', 338), 1),
 (('makes', '2257358', 411), 1),
 (('something', '30439611', 65), 1),
 (('plot', '12152908', 501), 1)]

In [0]:
print(ans)

[('death', ['Chargesheet', 'Stark Raving Mad', 'The Gaunt Stranger', 'Subconscious Cruelty', 'Benji', 'Quo Vadis, Baby?', 'Ghost From The Machine', 'Cherished', 'Aalkkoottathil Thaniye', "A Woman's Vengeance"])]


In [0]:
def text_match_cosine_similarity(query):
    query_final = []
    string_tokens = word_tokenize(query)
    query_final = [word.lower() for word in string_tokens if not word.lower() in stopwords.words('english')]
    query_final = sc.parallelize(query_final).map(lambda x: (x, 1))
    query_final = query_final.reduceByKey(lambda x, y: x+y)
    query_final = query_final.collect()
    q_size = len(query_final)
    
    for str in query_final:
        tf_q = str[1]/q_size
        idf_q = 1.405507153 #np.log()
        
        scount_d = word_count.filter(lambda x: x[0][0] == str)
        ndoc_d = scount.count()
        idf_d = np.log(float(total_doc)/ndoc_d)

        movies = scount_d.map(lambda x: [x[0][1], float(x[1]/x[0][2])*idf_d]).sortBy(lambda x: -x[1]).take(10)
    
    

In [0]:
text_match_cosine_similarity('The Man Who Came Back Man')

[('came', 1), ('man', 2), ('back', 1)]


In [0]:
import re
import math
import nltk
import string 
import numpy as np
import pandas as pd

from collections import Counter

from numpy import dot
from numpy.linalg import norm

from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Out[8]: True

In [0]:
def convert_to_vector_form(str):
    wordList = re.compile(r"\w+")
    all_applicable_words = wordList.findall(str)
    return Counter(all_applicable_words)

In [0]:
v = convert_to_vector_form("Machine explores the grey area bridge Machine")

In [0]:
#list(set(v))
v

Out[24]: ['Machine', 'explores', 'the', 'grey', 'area', 'bridge', 'Machine']

In [0]:
wordList

Out[13]: re.compile(r'\w+', re.UNICODE)